# Walmart Recruiting - Store Sales Forecasting -  Step 1: Data Discovery

Welocme to my series of notebooks to present my solution about this Kaggle competition.<br>
The objective is to give you a sense of my overall process rather than focusing too much on the LB score. Hope you will enjoy the reading. <br>Feel to contact me for more details

Given the historical sales data for 45 Walmart stores located in different regions. Each store contains a number of departments, and the task is to predict the department-wide sales for each store.

Also part of the challenge presented by this competition is modeling the effects of markdowns on these holiday weeks in the absence of complete/ideal historical data.

### Take away of the notebook
    . initial data profiling
    . merge all datasets together
    . save datasets to parquet format
    . Overall:
        +2 years of historical weekly sales (from week of 2010-02-05 to week of 2012-10-26)
        +1 year of weekly sales to predict (from week of 2012-11-02 to week of 2013-07-26)
        45 stores, 81 departements
        Seem to be some correlation between some features like CPI/unemployment 
        Too many missing values for the markdowns and it might not be meaningful enough to understand their
        potential effect on the weekly sales.

https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/data

## Quick glance before loading data into pandas df

Before loading anything into pandas dataframe, it is a good idea to quick glance at the files

#### list of files

In [2]:
!ls data

compressed           submission-2.csv     test.parquet
features.csv         submission-3.csv     train.csv
sampleSubmission.csv submission.csv       train.parquet
stores.csv           test.csv


#### number of rows per file

In [5]:
!wc -l data/*.csv

    8191 data/features.csv
  115065 data/sampleSubmission.csv
       0 data/stores.csv
  115065 data/test.csv
  421571 data/train.csv
  659892 total


#### display the first 5 rows of every csv file

In [6]:
!head -5 data/*.csv

==> data/features.csv <==
Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
1,2010-02-05,42.31,2.572,NA,NA,NA,NA,NA,211.0963582,8.106,FALSE
1,2010-02-12,38.51,2.548,NA,NA,NA,NA,NA,211.2421698,8.106,TRUE
1,2010-02-19,39.93,2.514,NA,NA,NA,NA,NA,211.2891429,8.106,FALSE
1,2010-02-26,46.63,2.561,NA,NA,NA,NA,NA,211.3196429,8.106,FALSE

==> data/sampleSubmission.csv <==
Id,Weekly_Sales
1_1_2012-11-02,0
1_1_2012-11-09,0
1_1_2012-11-16,0
1_1_2012-11-23,0

==> data/stores.csv <==
45,B,118221
==> data/test.csv <==
Store,Dept,Date,IsHoliday
1,1,2012-11-02,FALSE
1,1,2012-11-09,FALSE
1,1,2012-11-16,FALSE
1,1,2012-11-23,TRUE

==> data/train.csv <==
Store,Dept,Date,Weekly_Sales,IsHoliday
1,1,2010-02-05,24924.5,FALSE
1,1,2010-02-12,46039.49,TRUE
1,1,2010-02-19,41595.55,FALSE
1,1,2010-02-26,19403.54,FALSE


In [72]:
#### display the last 5 rows of every csv file

In [73]:
!tail -5 data/*.csv

==> data/baseline.csv <==
45_98_2013-06-28,690.52
45_98_2013-07-05,659.65
45_98_2013-07-12,695.21
45_98_2013-07-19,845.3
45_98_2013-07-26,657.63

==> data/features.csv <==
45,2013-06-28,76.05,3.639,4842.29,975.03,3,2449.97,3169.69,NA,NA,FALSE
45,2013-07-05,77.5,3.614,9090.48,2268.58,582.74,5797.47,1514.93,NA,NA,FALSE
45,2013-07-12,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,NA,NA,FALSE
45,2013-07-19,82.84,3.737,2961.49,1047.07,204.19,363,1059.46,NA,NA,FALSE
45,2013-07-26,76.06,3.804,212.02,851.73,2.06,10.88,1864.57,NA,NA,FALSE

==> data/sampleSubmission.csv <==
45_98_2013-06-28,0
45_98_2013-07-05,0
45_98_2013-07-12,0
45_98_2013-07-19,0
45_98_2013-07-26,0

==> data/stores.csv <==
45,B,118221Size
==> data/submission-2.csv <==
45_98_2013-06-28,6386.863739436284
45_98_2013-07-05,6386.863739436284
45_98_2013-07-12,6386.863739436284
45_98_2013-07-19,6386.863739436284
45_98_2013-07-26,6386.863739436284

==> data/submission-3.csv <==
45_98_2013-06-28,6419.6171875
45_98_2013-07-05,6543.528

#### Note:
Now that we have an idea of type of files we are dealing with (csv, header, num rows, separator) we can load the date into pandas dataframe quick safely

# loading data into pandas dataframes

#### installing pandas_profiling

In [4]:
# !pip3 install pandas_profiling

#### importing required packages

In [1]:
import pandas as pd
import pandas_profiling
from IPython.display import display

#### storing dataframes into a dictionary

In [19]:
dfs = {}
df_stores = pd.read_csv('data/stores.csv')
df_features = pd.read_csv('data/features.csv')
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
dfs = {'df_stores': df_stores,
       'df_features': df_features,
       'df_train': df_train,
       'df_test': df_test}

# initial data exploration

The general idea is to quickly develop an intuition of the data. I am not trying to get a complete a full EDA at this stage and to refine the data analysis,<br> I will proceed with an iterative approach.

#### dataframe shape

In [20]:
for k, df in dfs.items(): print('{} shape: {}'.format(k, df.shape))

df_stores shape: (45, 3)
df_features shape: (8190, 12)
df_train shape: (421570, 5)
df_test shape: (115064, 4)


#### List of columns

In [44]:
for k, df in dfs.items(): print(k), df.info(), print()

df_stores
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
Store    45 non-null int64
Type     45 non-null object
Size     45 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.1+ KB

df_features
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
Store           8190 non-null int64
Date            8190 non-null object
Temperature     8190 non-null float64
Fuel_Price      8190 non-null float64
MarkDown1       4032 non-null float64
MarkDown2       2921 non-null float64
MarkDown3       3613 non-null float64
MarkDown4       3464 non-null float64
MarkDown5       4050 non-null float64
CPI             7605 non-null float64
Unemployment    7605 non-null float64
IsHoliday       8190 non-null bool
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 711.9+ KB

df_train
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 col

#### Missing values

In [38]:
for k, df in dfs.items(): print('\n{}\n{}'.format(k, df.isnull().sum()))


df_stores
Store    0
Type     0
Size     0
dtype: int64

df_features
Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64

df_train
Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64

df_test
Store        0
Dept         0
Date         0
IsHoliday    0
dtype: int64


#### Unique values

In [37]:
for k, df in dfs.items(): print('\n{}\n{}'.format(k, df.nunique()))


df_stores
Store    45
Type      3
Size     40
dtype: int64

df_features
Store             45
Date             182
Temperature     4178
Fuel_Price      1011
MarkDown1       4023
MarkDown2       2715
MarkDown3       2885
MarkDown4       3405
MarkDown5       4045
CPI             2505
Unemployment     404
IsHoliday          2
dtype: int64

df_train
Store               45
Dept                81
Date               143
Weekly_Sales    359464
IsHoliday            2
dtype: int64

df_test
Store        45
Dept         81
Date         39
IsHoliday     2
dtype: int64


#### Initial stats / outliers for numerical features

In [36]:
for k, df in dfs.items(): print('\n{}\n{}'.format(k, df.describe()))


df_stores
           Store           Size
count  45.000000      45.000000
mean   23.000000  130287.600000
std    13.133926   63825.271991
min     1.000000   34875.000000
25%    12.000000   70713.000000
50%    23.000000  126512.000000
75%    34.000000  202307.000000
max    45.000000  219622.000000

df_features
             Store  Temperature   Fuel_Price      MarkDown1      MarkDown2  \
count  8190.000000  8190.000000  8190.000000    4032.000000    2921.000000   
mean     23.000000    59.356198     3.405992    7032.371786    3384.176594   
std      12.987966    18.678607     0.431337    9262.747448    8793.583016   
min       1.000000    -7.290000     2.472000   -2781.450000    -265.760000   
25%      12.000000    45.902500     3.041000    1577.532500      68.880000   
50%      23.000000    60.710000     3.513000    4743.580000     364.570000   
75%      34.000000    73.880000     3.743000    8923.310000    2153.350000   
max      45.000000   101.950000     4.468000  103184.980000  104

#### Initial stats  for categorical features

In [39]:
for k, df in dfs.items(): print('\n{}\n{}'.format(k, df.select_dtypes(['object', 'bool']).describe()))


df_stores
       Type
count    45
unique    3
top       A
freq     22

df_features
              Date IsHoliday
count         8190      8190
unique         182         2
top     2010-11-05     False
freq            45      7605

df_train
              Date IsHoliday
count       421570    421570
unique         143         2
top     2011-12-23     False
freq          3027    391909

df_test
              Date IsHoliday
count       115064    115064
unique          39         2
top     2012-12-21     False
freq          3002    106136


#### looking at the date stats

## Data profiling

In [41]:
for k, df in dfs.items(): print(k), display(pandas_profiling.ProfileReport(df))

df_stores


Number of variables,3
Number of observations,45
Total Missing (%),0.0%
Total size in memory,1.1 KiB
Average record size in memory,25.8 B
Numeric,2
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,0


df_features


Number of variables,12
Number of observations,8190
Total Missing (%),24.5%
Total size in memory,711.9 KiB
Average record size in memory,89.0 B
Numeric,10
Categorical,1
Boolean,1
Date,0
Text (Unique),0
Rejected,0


df_train


Number of variables,5
Number of observations,421570
Total Missing (%),0.0%
Total size in memory,13.3 MiB
Average record size in memory,33.0 B
Numeric,3
Categorical,1
Boolean,1
Date,0
Text (Unique),0
Rejected,0


df_test


Number of variables,4
Number of observations,115064
Total Missing (%),0.0%
Total size in memory,2.7 MiB
Average record size in memory,25.0 B
Numeric,2
Categorical,1
Boolean,1
Date,0
Text (Unique),0
Rejected,0


### date stats

In [69]:
### first conclusion
[print('{}\n{}\n'.format(k, pd.to_datetime(df[col]).describe()))
 for k, df in dfs.items() for col in df.columns if col=='Date']

df_features
count                    8190
unique                    182
top       2011-07-15 00:00:00
freq                       45
first     2010-02-05 00:00:00
last      2013-07-26 00:00:00
Name: Date, dtype: object

df_train
count                  421570
unique                    143
top       2011-12-23 00:00:00
freq                     3027
first     2010-02-05 00:00:00
last      2012-10-26 00:00:00
Name: Date, dtype: object

df_test
count                  115064
unique                     39
top       2012-12-21 00:00:00
freq                     3002
first     2012-11-02 00:00:00
last      2013-07-26 00:00:00
Name: Date, dtype: object



[None, None, None]

# combining all datasets together

It is usually easier to combine everything into a single dataset (if data size permitted) even iif you may endup
with a denormalized structure.

In [46]:
# combine train/test
%time merged = pd.concat([df_train, df_test], axis=0)
print(merged.shape)

# merge with store and features datasets
%time features = pd.merge(df_features, df_stores, on='Store', how='left')
print(features.Type.isnull().sum(), features.shape)

# merge train/test with features
%time merged = pd.merge(merged, features, on=['Store', 'Date', 'IsHoliday'], how='left')
print(merged.Type.isnull().sum(), merged.shape)

CPU times: user 13.7 ms, sys: 19.7 ms, total: 33.4 ms
Wall time: 38.8 ms
(536634, 5)
CPU times: user 5.33 ms, sys: 1.89 ms, total: 7.22 ms
Wall time: 8.3 ms
0 (8190, 14)
CPU times: user 118 ms, sys: 48.6 ms, total: 167 ms
Wall time: 183 ms
0 (536634, 16)


# save datasets

I found parquet format very interesting to super compress the data compare to other options such as pickle, feather so I will be using it.

In [12]:
# convert Date to datetime dataframe
merged['Date'] = pd.to_datetime(merged.Date)

# save train set
df = merged[:len(df_train)].reset_index(drop=True)
df.to_parquet('data/train.parquet', compression='brotli')

# save test set
df = merged[len(df_train):].reset_index(drop=True)
df.to_parquet('data/test.parquet', compression='brotli')